In [2]:
# отключим всякие предупреждения Anaconda
import warnings

warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import (GridSearchCV, cross_val_score,
                                     train_test_split)
from sklearn.preprocessing import StandardScaler

In [3]:
data = pd.read_csv('https://raw.githubusercontent.com/Yorko/mlcourse.ai/master/data/winequality-white.csv', sep=';')

In [4]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [6]:
X, y = data.drop('quality', axis = 1), data.quality

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, train_size=0.7, random_state=17) 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

# Линейная регрессия

In [11]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

LinearRegression()

In [14]:
print("Mean squared error (train): %.3f" %mean_squared_error(linreg.predict(X_train_scaled), y_train))
print("Mean squared error (test): %.3f" %mean_squared_error(linreg.predict(X_holdout_scaled), y_holdout))

Mean squared error (train): 0.558
Mean squared error (test): 0.584


In [22]:
linreg_coef = pd.DataFrame(np.abs(linreg.coef_), index= X.columns, columns=['importances'])

In [24]:
linreg_coef.sort_values('importances')

,importances
citric acid,0.000183
chlorides,0.008127
total sulfur dioxide,0.014304
free sulfur dioxide,0.042180
sulphates,0.062053
fixed acidity,0.097822
alcohol,0.129533
pH,0.150036
volatile acidity,0.192260
residual sugar,0.538164


# Lasso-регрессия

In [26]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, random_state=17)

In [27]:
lasso1_coef = pd.DataFrame(np.abs(lasso1.coef_), index= X.columns, columns=['importances'])

In [32]:
lasso1_coef.sort_values('importances')

,importances
fixed acidity,0.000000
citric acid,0.000000
total sulfur dioxide,0.000000
chlorides,0.002747
sulphates,0.029722
free sulfur dioxide,0.043088
pH,0.067277
volatile acidity,0.188479
density,0.235492
residual sugar,0.256363


In [44]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=17)
lasso_cv.fit(X_train_scaled, y_train);

In [45]:
lasso_cv.alpha_

0.0002833096101839324

In [47]:
lasso_cv_coef = pd.DataFrame(np.abs(lasso_cv.coef_), index= X.columns, columns=['importances'])
lasso_cv_coef.sort_values('importances')

,importances
citric acid,0.000000
chlorides,0.006933
total sulfur dioxide,0.012969
free sulfur dioxide,0.042698
sulphates,0.060939
fixed acidity,0.093295
alcohol,0.137115
pH,0.146549
volatile acidity,0.192049
residual sugar,0.526883


In [49]:
print("Mean squared error (train): %.3f" % mean_squared_error(lasso_cv.predict(X_train_scaled), y_train))
print("Mean squared error (test): %.3f" % mean_squared_error(lasso_cv.predict(X_holdout_scaled), y_holdout))

Mean squared error (train): 0.558
Mean squared error (test): 0.583


# Случайный лес

In [55]:
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train_scaled, y_train)

RandomForestRegressor(random_state=17)

In [62]:
print("Mean squared error (train): %.3f" % mean_squared_error(forest.predict(X_train_scaled), y_train))
print("Mean squared error (cv): %.3f" %np.mean(cross_val_score(forest, X_train_scaled, y_train, scoring='neg_mean_squared_error')))
print("Mean squared error (test): %.3f" % mean_squared_error(forest.predict(X_holdout_scaled), y_holdout))

Mean squared error (train): 0.053
Mean squared error (cv): -0.414
Mean squared error (test): 0.372


In [63]:
forest_params = {'max_depth': list(range(10, 25)), 
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6,12))}

locally_best_forest = GridSearchCV(forest, forest_params, cv=5)
locally_best_forest.fit(X_train_scaled, y_train)

KeyboardInterrupt: 

In [ ]:
locally_best_forest.best_params_, locally_best_forest.best_score_

In [66]:
rf = RandomForestRegressor(max_depth=19, max_features=7, min_samples_leaf=1 , random_state=17)
rf.fit(X_train_scaled, y_train)

RandomForestRegressor(max_depth=19, max_features=7, random_state=17)

In [72]:
print("Mean squared error (cv): %.3f" % np.mean(np.abs(cross_val_score(rf, X_train_scaled, y_train, scoring='neg_mean_squared_error'))))
print("Mean squared error (test): %.3f" % mean_squared_error(forest.predict(X_holdout_scaled), y_holdout))

Mean squared error (cv): 0.404
Mean squared error (test): 0.372


In [70]:
rf_importance = pd.DataFrame(rf.feature_importances_, index=X.columns, columns=['importances'])

In [71]:
rf_importance.sort_values('importances')

,importances
sulphates,0.059499
citric acid,0.061500
fixed acidity,0.061873
chlorides,0.067819
total sulfur dioxide,0.071644
residual sugar,0.072529
pH,0.072646
density,0.086530
free sulfur dioxide,0.113509
volatile acidity,0.117627
